In [1]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validation = {
    "week1": [
        "/kaggle/input/new-raw/clean_week1/val/val_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw/clean_week2/val/val_week1_2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw/clean_week3/val/val_week1_2_3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw/clean_week4/val/val_week1_2_3_4.csv"
    ]
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw/clean_week4/test/test_week4.csv"
    ]
}

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

## Tìm siêu tham số tốt nhất cho từng tuần

In [3]:
# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validation):
       # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validation)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    model = KNeighborsClassifier()

    param_grid = {
        'n_neighbors': [3, 5, 7, 9],  # Số lượng hàng xóm gần nhất
        'weights': ['uniform', 'distance'],  # Trọng số cho hàng xóm
        'metric': ['euclidean', 'manhattan', 'minkowski']  # Khoảng cách sử dụng
    }
    # param_grid = {
    #     'n_neighbors': [3],  # Số lượng hàng xóm gần nhất
    #     'weights': ['uniform'],  # Trọng số cho hàng xóm
    #     'metric': ['euclidean']  # Khoảng cách sử dụng
    # }
    # Khởi tạo RandomSearch tuner
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, scoring='accuracy',
                              verbose=2, n_jobs=-1)
        
    
   # Huấn luyện mô hình với các siêu tham số tốt nhất
    grid_search.fit(X_train_res, y_train_res)

    # Đánh giá mô hình
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_val)
    # Trả về kết quả tối ưu cho tuần
    best_params = grid_search.best_params_
    return best_params

In [4]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    "week1": "/kaggle/input/new-raw/clean_week1/train/clean_data_week1.csv",
    "week2": "/kaggle/input/new-raw/clean_week2/train/clean_data_week2.csv",
    "week3": "/kaggle/input/new-raw/clean_week3/train/clean_data_week3.csv",
    "week4": "/kaggle/input/new-raw/clean_week4/train/clean_data_week4.csv"
}

file_validation = {
    "week1": "/kaggle/input/new-raw/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw/clean_week4/val/val_week1_2_3_4.csv"
}

In [5]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name, param_value in best_params_week1.items():
       print(f"{param_name}: {param_value}")

print("\nBest Parameters for Week 2:")
for param_name, param_value in best_params_week2.items():
       print(f"{param_name}: {param_value}")

print("\nBest Parameters for Week 3:")
for param_name, param_value in best_params_week3.items():
       print(f"{param_name}: {param_value}")

print("\nBest Parameters for Week 4:")
for param_name, param_value in best_params_week4.items():
       print(f"{param_name}: {param_value}")

Fitting 10 folds for each of 24 candidates, totalling 240 fits
Fitting 10 folds for each of 24 candidates, totalling 240 fits
Fitting 10 folds for each of 24 candidates, totalling 240 fits
Fitting 10 folds for each of 24 candidates, totalling 240 fits
Best Parameters for Week 1:
metric: manhattan
n_neighbors: 5
weights: distance

Best Parameters for Week 2:
metric: manhattan
n_neighbors: 3
weights: distance

Best Parameters for Week 3:
metric: manhattan
n_neighbors: 3
weights: distance

Best Parameters for Week 4:
metric: manhattan
n_neighbors: 3
weights: distance


In [6]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

## KNN với các tham sốt nhất cho mỗi tuần

In [7]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week]
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = train_data['classification_encoded']
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = test_data['classification_encoded']


        # Xây dựng mô hình với tham số tốt nhất
        model = KNeighborsClassifier(**params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        accuracy = accuracy_score(y_test, y_pred)
        week_results["accuracy_per_fold"].append(accuracy)
        # Dự đoán

        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix|
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
Fold 1: Using file /kaggle/input/new-raw/clean_week1/train/5-folds/data_part_1.csv as test set
Fold 2: Using file /kaggle/input/new-raw/clean_week1/train/5-folds/data_part_2.csv as test set
Fold 3: Using file /kaggle/input/new-raw/clean_week1/train/5-folds/data_part_3.csv as test set
Fold 4: Using file /kaggle/input/new-raw/clean_week1/train/5-folds/data_part_4.csv as test set
Fold 5: Using file /kaggle/input/new-raw/clean_week1/train/5-folds/data_part_5.csv as test set

=== Average Precision, Recall, F1-Score per Label ===
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.752412        0.826000          0.787394
1      1           0.638690        0.538871          0.583361
2      2           0.685275        0.563230          0.615914
3      3           0.716783        0.655090          0.683605
4      4           0.91243

## Kết quả 5-folds

In [8]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 0.0153 seconds
Average Test Time: 0.0454 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.752412        0.826000          0.787394
1      1           0.638690        0.538871          0.583361
2      2           0.685275        0.563230          0.615914
3      3           0.716783        0.655090          0.683605
4      4           0.912435        0.910814          0.911600

Average Confusion Matrix:
       0     1     2      3       4
0  495.6  12.6  15.8    5.4    70.6
1   23.8  47.2   3.6    3.4     9.6
2   38.8   4.8  92.6    6.4    21.8
3   12.0   2.8   4.6  109.4    38.2
4   88.6   7.0  19.2   28.2  1460.4

=== Results for week2 ===
Average Train Time: 0.0032 seconds
Average Test Time: 0.3632 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.765598        

## Test trên tập test

In [9]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[week]
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Xây dựng mô hình với tham số tốt nhất
    model = KNeighborsClassifier(**params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test), columns=np.unique(y_test)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [10]:
process_week(1, best_params, results)


=== Processing Week 1 ===

=== Precision, Recall, F1-Score per Label ===
   Label  Precision    Recall  F1-Score
0      0   0.729730  0.792000  0.759591
1      1   0.547170  0.537037  0.542056
2      2   0.610526  0.563107  0.585859
3      3   0.578947  0.628571  0.602740
4      4   0.915551  0.886341  0.900709

=== Confusion Matrix ===
     0   1   2   3    4
0  297   9  16  11   42
1   17  29   3   1    4
2   27   2  58   4   12
3   11   2   2  66   24
4   55  11  16  32  889

Train Time: 0.02 seconds
Test Time: 0.03 seconds


In [11]:
process_week(2, best_params, results)


=== Processing Week 2 ===

=== Precision, Recall, F1-Score per Label ===
   Label  Precision    Recall  F1-Score
0      0   0.748130  0.800000  0.773196
1      1   0.563636  0.574074  0.568807
2      2   0.614583  0.572816  0.592965
3      3   0.580952  0.580952  0.580952
4      4   0.918616  0.900299  0.909366

=== Confusion Matrix ===
     0   1   2   3    4
0  300  11  14   9   41
1   15  31   3   1    4
2   24   4  59   5   11
3    9   3   8  61   24
4   53   6  12  29  903

Train Time: 0.00 seconds
Test Time: 0.28 seconds


In [12]:
process_week(3, best_params, results)


=== Processing Week 3 ===

=== Precision, Recall, F1-Score per Label ===
   Label  Precision    Recall  F1-Score
0      0   0.748111  0.792000  0.769430
1      1   0.517241  0.555556  0.535714
2      2   0.552083  0.514563  0.532663
3      3   0.627451  0.609524  0.618357
4      4   0.927052  0.912263  0.919598

=== Confusion Matrix ===
     0   1   2   3    4
0  297  14  18  10   36
1   17  30   3   1    3
2   26   5  53   7   12
3   10   3   7  64   21
4   47   6  15  20  915

Train Time: 0.00 seconds
Test Time: 0.36 seconds


In [13]:
process_week(4, best_params, results)


=== Processing Week 4 ===

=== Precision, Recall, F1-Score per Label ===
   Label  Precision    Recall  F1-Score
0      0   0.751880  0.800000  0.775194
1      1   0.509804  0.481481  0.495238
2      2   0.585106  0.533981  0.558376
3      3   0.659794  0.609524  0.633663
4      4   0.926927  0.923230  0.925075

=== Confusion Matrix ===
     0   1   2   3    4
0  300  12  18  12   33
1   18  26   5   1    4
2   25   5  55   5   13
3    9   3   6  64   23
4   47   5  10  15  926

Train Time: 0.00 seconds
Test Time: 0.45 seconds


In [14]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 0.02 seconds
  Test Time: 0.03 seconds
  Precision: [0.72972973 0.54716981 0.61052632 0.57894737 0.91555098]
  Recall: [0.792      0.53703704 0.5631068  0.62857143 0.88634098]
  F1-Score: [0.75959079 0.54205607 0.58585859 0.60273973 0.90070922]
  Confusion Matrix:
[[297   9  16  11  42]
 [ 17  29   3   1   4]
 [ 27   2  58   4  12]
 [ 11   2   2  66  24]
 [ 55  11  16  32 889]]
Week 2:
  Train Time: 0.00 seconds
  Test Time: 0.28 seconds
  Precision: [0.74812968 0.56363636 0.61458333 0.58095238 0.91861648]
  Recall: [0.8        0.57407407 0.57281553 0.58095238 0.9002991 ]
  F1-Score: [0.77319588 0.56880734 0.59296482 0.58095238 0.90936556]
  Confusion Matrix:
[[300  11  14   9  41]
 [ 15  31   3   1   4]
 [ 24   4  59   5  11]
 [  9   3   8  61  24]
 [ 53   6  12  29 903]]
Week 3:
  Train Time: 0.00 seconds
  Test Time: 0.36 seconds
  Precision: [0.74811083 0.51724138 0.55208333 0.62745098 0.92705167]
  Recall: [0.792      0.